## 講習4 --- Numpyの基本 

この次に、IRAFを使わずにFITSファイルの処理をする方法を扱いますが、その準備としてNumpyの基本を知っておきましょう。  

Numpyは、数値計算を効率よく処理するためのサードパーティモジュールです。  特に、多次元配列を取り扱う際に処理速度が速くなり、コードの表記も効率的になります。
pythonの標準の配列であるリスト型では処理が遅いため、科学計算ではNumpyのndarrayという多次元配列のデータ型を使います。Numpyはサードパーティモジュールですが、科学計算では標準的に使われます。このあと紹介するastropy.io.fitsおよびmatplotlibでも、このndarrayを採用しています。  

以下、ndarrayの基本について説明し、numpyの基本的で使える関数について説明します。 

### リスト型 
まずはpythonの標準のリスト型を見てみましょう。


In [1]:
a = [1, 2, 3, 4]
b = [10, 20, 30, 40]

\+ 演算子はリストとリストの結合になります。\* 演算子はリストの繰り返しを作成します。

In [2]:
a + b

[1, 2, 3, 4, 10, 20, 30, 40]

In [3]:
a * 4

[1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]

ベクトル的に演算したい場合には、下のように、forループを回して各要素を取り出してから演算をする必要があります。

In [4]:
n = len(a)
c = [0] * n
for i in range(n):
    c[i] = a[i] + b[i]
c

[11, 22, 33, 44]

In [5]:
print (c)

[11, 22, 33, 44]


### Numpyのndarray    
Numpyではもっとすっきりした処理、**ベクトル処理**、ができます。処理速度も速いです。

In [6]:
import numpy as np   #  一般的に np と省略される

numpy.array() 関数を使って、pythonのリストをndarray に変換します。

In [7]:
an = np.array(a)  
bn = np.array(b)

In [8]:
cn = an + bn
cn

array([11, 22, 33, 44])

In [9]:
print (cn)

[11 22 33 44]


In [10]:
an * 4

array([ 4,  8, 12, 16])

### numpyの関数 (1)  ~ 基本 + いくつか  
numpyで用意されている関数は数多くあります。 cellの中で、np.とタイプしてタブキーを押すと候補が表示されます。あるいはnp.mでタブキーを押すとmで始まる関数の候補が表示されます。

In [11]:
np.max(cn), np.min(cn), np.mean(cn)

(44, 11, 27.5)

#### numpy.where()    
天文のデータ処理では(でなくても?)次の numpy.where( ) が便利です。 バッドマスク処理や外れ値の除外に使えます。

In [12]:
dn = np.array([1, 1, 3, 1, 1])

**np.where( )** 関数は、( )内の条件を満たす要素のindexを返します。 下の例では、2以上の値をとる要素に-99を代入するという意味です。

In [13]:
dn[np.where(dn > 2)] = -99   

In [14]:
dn

array([  1,   1, -99,   1,   1])

#### numpy.zeros( )   
下のように、**numpy.zeros( )** を使って、全要素が0の5 x 5 の二次元配列を作成することができます。  

In [15]:
data = np.zeros((5, 5))

In [16]:
data

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [17]:
data[0, 1] = 2   #   [0, 1]の要素に2を代入する。  

numpyに限らず、pythonではindexは0から始まります。C言語と同じです。

In [18]:
data

array([[ 0.,  2.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

このdataの表示を見て気づいたかとも思いますが、
左下を原点とした二次元画像としたときに、ndarrayでは **(y, x)**のように xとyのindexを逆にして要素が格納されます。
後で、astropy.io.fitsのところでも再び触れます。

### numpyの関数 (2) ~ 画像重ね合わせの準備  

IRAFのimcombineを使わずに、astropy.io.fits + numpyで重ね合せる( つまり、複数画像の各ピクセルでのaverageやmedianを求めてcombineをする)ための準備として、numpy.stack(), numpy.average(), numpy.median() などを紹介します。  

####  1次元
まずは1次元配列で見てみましょう。  
3つの1次元配列、a, b, cがあるとき、

In [19]:
a1 = np.array([1, 2, 3])
b1 = np.array([2, 3, 4])
c1 = np.array([2, 2, 5])

**numpy.stack()**関数を使うことでこれらをまとめて一つ次元の高い配列にする(スタックする)ことができます。 

In [20]:
s1 = np.stack((a1, b1, c1))

In [21]:
s1

array([[1, 2, 3],
       [2, 3, 4],
       [2, 2, 5]])

そのうえで、配列を重ねてできた新しい次元の軸に沿ってmedianをとります。  
上の表示例だと、縦方向にmedianをとります。  
具体的には、**np.median([スタックされた配列], axis=0)** としてmedianの配列を得ます。

In [22]:
meddata = np.median(s1, axis=0)

In [23]:
print(meddata)

[ 2.  2.  4.]


averageやsumも同様です。

In [24]:
print (np.average(s1, axis=0))

[ 1.66666667  2.33333333  4.        ]


In [25]:
print (np.sum(s1, axis=0))

[ 5  7 12]


axis=0を省略すると全ての要素について計算します。  

In [26]:
print (np.average(s1))

2.66666666667


#### 2次元

2次元の場合も同様です。

In [27]:
a2 = np.array([[1, 2, 1],[20, 21, 13]])
b2 = np.array([[2, 2, 2],[22, 23, 15]])

In [28]:
print (a2)
print ()
print (b2)
print ()
print (np.average(np.stack((a2, b2)), axis=0))

[[ 1  2  1]
 [20 21 13]]

[[ 2  2  2]
 [22 23 15]]

[[  1.5   2.    1.5]
 [ 21.   22.   14. ]]


(次に紹介する) astropy.io.fitsで読み込んだ画像の場合も同様にスタックをしてmedianやaverageをとります。   

#### np.append() 

最初からスタックする配列がそろっていれば上のように**np.stack()**を使うのが簡単ですが、通常のデータ処理の場合には、リストから次々にファイルを読みこんでスタックに追加していくことでより効率よくプログラミングができます。  
次々にファイルを読み込んでスタックに追加する場合には、**np.append()**を使用します。

In [30]:
s2 = a1[np.newaxis, :]
s2 = np.append(s2, b1[np.newaxis, :], axis=0)
s2 = np.append(s2, c1[np.newaxis, :], axis=0)

In [31]:
s2

array([[1, 2, 3],
       [2, 3, 4],
       [2, 2, 5]])

上で求めたs1と同じですね。

### numpyの関数 (3) ~ テキストデータの読み込み

Numpyがなくても、Pythonの標準機能でテキストファイルの読み書きはできます。まずはその方法を紹介します。
./sample_data/mag.txt はテキストファイルで、等級(列1)、等級エラー(列2)のテーブルが記録されています。

In [32]:
mag = []   #   空のリストを準備
merr = []
f = open('./sample_data/mag.txt')
for line in f:
    v = line.rstrip().split() # 末尾の改行コードを削除して、空白文字で分割
    mag.append(float(v[0]) )  # リストに要素を追加、文字列をfloatに変換 
    merr.append(float(v[1]))
f.close()

f = open() と f.close() の組み合わせは、Pythonの教科書でまず出てくるファイルの入出力時に使う構文です。  
しかし、f = open() と f.close()の間で例外処理が行われたときに、close()されないという弊害が起こりますので、
安全なwith open() の構文をオススメします。例外処理が行われたときに自動的にclose()されます。


In [33]:
mag = []   #   空のリストを準備
merr = []
with open('./sample_data/mag.txt') as f:
    for line in f:
        v = line.rstrip().split() # 末尾の改行コードを削除して、空白文字で分割
        mag.append(float(v[0]) )  # リストに要素を追加、文字列をfloatに変換 
        merr.append(float(v[1]))

In [34]:
mag[:5]   #   長いリストなので最初の5つだけ抽出

[15.025, 16.236, 17.97, 17.677, 19.086]

In [35]:
merr[:5]

[0.004, 0.011, 0.053, 0.049, 0.16]

数値テーブルであることが分かっていれば、**numpy.loadtxt()**を使ってもっと簡単に読み込み、ndarrayに保存することができます。

In [ ]:
np.set_printoptions(precision=3, suppress=True)  #  suppress=Trueで指数表示禁止。この行は必須ではない。

In [36]:
mlist = np.loadtxt('./sample_data/mag.txt')

In [37]:
mlist

array([[  1.50250000e+01,   4.00000000e-03],
       [  1.62360000e+01,   1.10000000e-02],
       [  1.79700000e+01,   5.30000000e-02],
       ..., 
       [  1.62160000e+01,   1.20000000e-02],
       [  1.89290000e+01,   1.56000000e-01],
       [  1.85660000e+01,   1.30000000e-01]])

In [ ]:
np.savetxt('result1calib.txt', mlist, fmt='%.3f')

array([[等級, 等級エラー], [等級, 等級エラー], .... ] )  

**numpy.loadtxt()**のオプションにて、ヘッダ行を無視、区切り文字(カンマ区切りやタブ区切り)、どの列を読み込むか、などを指定することができます。

### ブロードキャスティング  

ある2次元配列があるとき、そのどちらかの軸の大きさと同じ大きさの1次元配列の足し算、引き算をすると、もう一つの軸に沿って同じ演算をしてくれます。  

In [38]:
data1 = np.array([0, 1, 2])
data2 = np.array([[0, 0, 0], [10, 10, 10], [20, 20, 20], [30, 30, 30]])

In [39]:
data2 + data1 

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

<img src='./img/broadcasting.png', width='500'> 

In [40]:
data3 = np.array([0, 100, 200])
data2 + data3

array([[  0, 100, 200],
       [ 10, 110, 210],
       [ 20, 120, 220],
       [ 30, 130, 230]])

次のastropy.io.fitsのところで、バイアス値を引くケースで使います。

### 配列について補足  
Pythonの配列には、**リスト**と**タプル**があります。  
**リスト**は [ ] で囲まれ変更**可能**なオブジェクトです。  
**タプル**は( ) で囲まれ変更**不可能**なオブジェクトです。


In [41]:
aa = [1, 2, 3, 4]

In [42]:
aa[2] = 10

In [43]:
aa

[1, 2, 10, 4]

In [44]:
bb = (10, 20, 30, 40)

In [45]:
bb

(10, 20, 30, 40)

In [46]:
bb[2] = 100

TypeError: 'tuple' object does not support item assignment

タプルの要素を書き換えようとしたら怒られました。    